In [1]:
from collections import defaultdict
import gensim
import pandas as pd
from spacy.lang.en import English
nlp = English()

/opt/conda/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/train.csv")
all_document = train_df["comment_text"]+test_df["comment_text"]
# kaggle だとメモリ足らない
del train_df, test_df

#nlp の箇所は word.split() や nltk.word_torknize でも可
#その場合は sentences = [line.split() for line in all_document]
#token.text を使えば original text が入手可能
#lemma:見出し語 pos_:品詞 dep_:依存関係
document_info = [[token.lemma, token.pos_, token.dep_] for line in all_document for token in nlp(line)]
document_info [:5]
#list を転置して lemma のみ取得
sentences = list(map(list, zip(*document_info)))
sentences[:5]
word_count = defaultdict(lambda: 0)
for line in sentences:
    for word in line:
        word_count[word] += 1
word_count
sentences =  [token[0] if word_count[token[0]] >= MIN_COUNT token[1] for line in document_info for token in line]

In [4]:
# line.split() はnltk.word_torknize(line) でも可
sentences = [line.split() for line in all_document]

In [5]:
MIN_COUNT = 10

In [6]:
word2vec = gensim.models.Word2Vec(sentences, window=5, min_count=MIN_COUNT , iter=5, negative=5)

In [7]:
word2vec.save("jigsaw_word2vec")

In [10]:
word2vec.wv.most_similar("suck")

[('screw', 0.7839298844337463),
 ('soak', 0.7812076807022095),
 ('blow', 0.761015772819519),
 ('gobble', 0.7414670586585999),
 ('lighten', 0.7212039232254028),
 ('pick', 0.6987291574478149),
 ('Suck', 0.6884019374847412),
 ('grow', 0.6851153373718262),
 ('crank', 0.6800088882446289),
 ('sucking', 0.6752239465713501)]